In [2]:
import os
import sys
import time
import yaml
import pandas as pd
import numpy as np

with open('../../config.local.yaml', 'r') as f:
    local_config = yaml.safe_load(f)

LOCAL_PATH = local_config['LOCAL_PATH']

sys.path.append(os.path.join(LOCAL_PATH, "src/python"))

import data_tools as dt


In [4]:
agenda_df = dt.get_agenda_items()
minutes_df = dt.get_minutes()

No data found for 2021-01-14
No data found for 2022-03-17
No data found for 2022-10-13


In [12]:
"""
Check if each agenda item with a casenum is contained in the minutes
"""
df = agenda_df.loc[
    agenda_df['is_casenum'], 
    ['year', 'date', 'item_no', 'title', 'is_casenum']
].reset_index(drop=True)

df = df.merge(
    minutes_df[['year', 'date', 'item_no', 'project_result']],
    on = ['year', 'date', 'item_no'],
    how = 'left',
).reset_index(drop=True)

assert df['project_result'].isna().sum() == 0